<a href="https://colab.research.google.com/github/manwanth/transductive-transfer-learning-model/blob/main/dataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import gspread
from google.colab import auth
from google.auth.transport.requests import Request
from google.auth import default
from google.colab import drive
import matplotlib.font_manager as fm
# Make sure to use fonts that differ by some margin as to keep input and target domains separate
# AUTHENTICATE TO GOOGLE
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# PARAMETERS
# Find a font available in the Colab environment
# You can list available fonts with fm.findSystemFonts(fontpaths=None, fontext='ttf')
font_path = fm.findfont(fm.FontProperties(family='DejaVu Sans', style='italic', weight='light'))
image_size = (16, 16)
font_size = 10
digits = list(range(10))
num_images_per_digit = 20  # samples per digit
sheet_name = "Digit_16x16_Dataset"

# GENERATE DATA
font = ImageFont.truetype(font_path, font_size)
data_rows = []

for digit in digits:
    for _ in range(num_images_per_digit):
        img = Image.new("L", image_size, color=0)  # grayscale
        draw = ImageDraw.Draw(img)

        # Get the size of the rendered digit
        bbox = draw.textbbox((0, 0), str(digit), font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        # Calculate top-left coordinates to center the digit
        x = (image_size[0] - text_width) // 2
        y = (image_size[1] - text_height) // 2

        draw.text((x, y), str(digit), font=font, fill=255)

        # Convert to numpy array
        img_array = np.array(img, dtype=np.float32) / (255.0/2) - 1

        # Add Gaussian noise
        noise = np.random.normal(0, 0.05, img_array.shape)  # mean=0, std=0.05
        img_array = np.clip(img_array + noise, 0, 1)  # keep in [0, 1]

        # Flatten and append label
        row = list(img_array.flatten()) + [digit]
        data_rows.append(row)

# SAVE TO GOOGLE SHEETS
df = pd.DataFrame(data_rows)
df.columns = [f"pixel_{i+1}" for i in range(16*16)] + ["label"]

# Create a new sheet and upload
spreadsheet = gc.create(sheet_name)
worksheet = spreadsheet.get_worksheet(0)
worksheet.update([df.columns.tolist()] + df.values.tolist())

print(f"Data uploaded to Google Sheet: {sheet_name}")